### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold

%matplotlib inline

### Load Iris Data

In [2]:
bc = load_breast_cancer()
X = bc.data
y = bc.target

In [6]:
print(bc.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

### Display Data

In [7]:
X[:3]

array([[1.799e+01, 1.038e+01, 1.228e+02, 1.001e+03, 1.184e-01, 2.776e-01,
        3.001e-01, 1.471e-01, 2.419e-01, 7.871e-02, 1.095e+00, 9.053e-01,
        8.589e+00, 1.534e+02, 6.399e-03, 4.904e-02, 5.373e-02, 1.587e-02,
        3.003e-02, 6.193e-03, 2.538e+01, 1.733e+01, 1.846e+02, 2.019e+03,
        1.622e-01, 6.656e-01, 7.119e-01, 2.654e-01, 4.601e-01, 1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, 1.326e+03, 8.474e-02, 7.864e-02,
        8.690e-02, 7.017e-02, 1.812e-01, 5.667e-02, 5.435e-01, 7.339e-01,
        3.398e+00, 7.408e+01, 5.225e-03, 1.308e-02, 1.860e-02, 1.340e-02,
        1.389e-02, 3.532e-03, 2.499e+01, 2.341e+01, 1.588e+02, 1.956e+03,
        1.238e-01, 1.866e-01, 2.416e-01, 1.860e-01, 2.750e-01, 8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, 1.203e+03, 1.096e-01, 1.599e-01,
        1.974e-01, 1.279e-01, 2.069e-01, 5.999e-02, 7.456e-01, 7.869e-01,
        4.585e+00, 9.403e+01, 6.150e-03, 4.006e-02, 3.832e-02, 2.058e-02,
        2.250e-02, 4.571e-03, 2.357e

In [9]:
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Split Data

Below, we are splitting the data into a train and test or holdout set. Only use the test set to evaluate your final model.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.33, random_state=42, stratify=y)

### Verifying Class Percentages

Notice that the percentage of class 0, 1, 2 are the same in the original datset as in the split set.

In [11]:
pd.Series(y).value_counts() / y.shape[0]

1    0.627417
0    0.372583
dtype: float64

In [12]:
pd.Series(y_train).value_counts() / y_train.shape[0]

1    0.627297
0    0.372703
dtype: float64

### KFold

Here, we are building 3 KNN models

- a model with k=3
- a model with k=11
- a model with k=21

Each model is trained 5 times. The training happens on 4 subsets and evaluation happens on the 5th subset. We then take the average of all 5 runs and average them into a single value. This value represents the mean accuracy of the KNN at a particular "k" value.

In [13]:
kf = KFold(n_splits=5)
scores = np.zeros(3)
params = [3, 11, 21]
for i, k in enumerate(params):
    knn = KNeighborsClassifier(n_neighbors=k)
    temp = np.zeros(5)
    for j, (train_index, val_index) in enumerate(kf.split(X_train)):
        X_tr = X_train[train_index]
        y_tr = y_train[train_index]
        X_vl = X_train[val_index]
        y_vl = y_train[val_index]
    
        knn.fit(X_tr, y_tr)
        accuracy = knn.score(X_vl, y_vl)
        temp[j] = accuracy
        
    scores[i] = temp.mean()
print(scores)

[0.93701299 0.93964457 0.93174983]


### Choosing the best hyperparameter

Now that we have 3 average accuracy scores, we can find the hyperparameter that performed the best, i.e., had the highest accuracy.

In [14]:
best = params[scores.argmax()]; best

11

### Final Model

We now have the best hyperparameter. So we build the final model on all the training data and evaluate it on the test or holdout set to see how the model performs on data it has never seen.

In [15]:
final = KNeighborsClassifier(n_neighbors=best).fit(X_train, y_train)
final.score(X_test, y_test)

0.9361702127659575